In [1]:
from pymatgen.core import Structure, Lattice
from pymatgen.entries.computed_entries import ComputedStructureEntry
import json 
import pandas as pd
from tqdm import tqdm
import os

## Alexandria

---

In [ ]:
import os
import shutil

def move_files(source_directory, destination_directory):
    os.makedirs(destination_directory, exist_ok=True)

    for root, dirs, files in os.walk(source_directory):
        for file in files:
            source_file_path = os.path.join(root, file)
            destination_file_path = os.path.join(destination_directory, file)

            shutil.move(source_file_path, destination_file_path)
            print(f"Перемещен файл: {source_file_path} -> {destination_file_path}")

source_directory = 'alexandria\\1'
destination_directory = 'alexandria\\'

move_files(source_directory, destination_directory)


Перемещен файл: alexandria\1\alexandria_006.json\alexandria_006.json -> alexandria\alexandria_006.json
Перемещен файл: alexandria\1\alexandria_007.json\alexandria_007.json -> alexandria\alexandria_007.json
Перемещен файл: alexandria\1\alexandria_008.json\alexandria_008.json -> alexandria\alexandria_008.json
Перемещен файл: alexandria\1\alexandria_009.json\alexandria_009.json -> alexandria\alexandria_009.json
Перемещен файл: alexandria\1\alexandria_010.json\alexandria_010.json -> alexandria\alexandria_010.json
Перемещен файл: alexandria\1\alexandria_011.json\alexandria_011.json -> alexandria\alexandria_011.json
Перемещен файл: alexandria\1\alexandria_012.json\alexandria_012.json -> alexandria\alexandria_012.json
Перемещен файл: alexandria\1\alexandria_013.json\alexandria_013.json -> alexandria\alexandria_013.json
Перемещен файл: alexandria\1\alexandria_014.json\alexandria_014.json -> alexandria\alexandria_014.json
Перемещен файл: alexandria\1\alexandria_015.json\alexandria_015.json -> a

In [2]:
def open_json(path: str):
    with open(path) as file:
        data = json.load(file)
    return data['entries']

In [3]:
def from_entry_to_cif_energy(data): 
    lattice = Lattice(data['structure']['lattice']['matrix'])

    species = []
    coords = []
    for site in data['structure']['sites']:
        for specie in site['species']:
            species.append(specie['element'])
            coords.append(site['xyz'])

    structure = Structure(lattice, species, coords)

    entry = ComputedStructureEntry(
        structure=structure,
        energy=data['energy'],
        correction=data['correction'],
        entry_id=data['entry_id'],
        parameters=data['parameters'],
        composition=data['composition'],
        data=data['data']
    )
    cif_string = structure.to(fmt='cif')
    
    return cif_string, entry.data

In [4]:
def create_dataframe(files: list, path_to_callback: str):
    cifs_for_dataframe = []
    data_for_dataframe = []
    for file in tqdm(files):
        entrys = open_json(file)
        cifs_for_callback = []
        data_for_callback = []
        
        for entry in entrys:
            cif, entry_data = from_entry_to_cif_energy(entry)
            cifs_for_callback.append(cif)
            data_for_callback.append(entry_data)
        
        callback_df = pd.concat([pd.DataFrame(data_for_callback), pd.DataFrame({"cif": cifs_for_callback})], axis=1)
        callback_df.to_csv(path_to_callback + os.path.basename(file)[:-5] + ".csv")
        
        data_for_dataframe += data_for_callback
        cifs_for_dataframe += cifs_for_callback
            
    df_entry_data = pd.DataFrame(data_for_dataframe)
    df_cif = pd.DataFrame({"cif": cifs_for_dataframe})
    
    final_df = pd.concat([df_entry_data, df_cif], axis=1)
    
    return final_df            
        

In [ ]:
directory_path = 'alexandria\\jsons\\'

files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

In [6]:
alexandria_dataframe = create_dataframe(["D:\Projects\projects_python\genCry\\alexandria\convex_hull_pbe.json"], "alexandria\\callbacks\\")

  0%|          | 0/1 [00:00<?, ?it/s]d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\composition.py:1337: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\composition.py:366: UserWarning: No Pauling electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda sym: get_el_sp(sym).X)
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\composition.py:1337: UserWarning: No Pauling electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, 

In [ ]:
directory_path = 'alexandria\callbacks'
output_file_path = 'alexandria\\alexandria_full.csv'

csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

dataframes = []

for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)  
    df = pd.read_csv(file_path)
    dataframes.append(df)  

combined_df = pd.concat(dataframes, ignore_index=True)

combined_df.to_csv(output_file_path, index=False)

print(f"Все файлы объединены в: {output_file_path}")

Все файлы объединены в: alexandria\alexandria_full.csv


In [65]:
data = pd.read_csv("alexandria\\alexandria_full.csv")

In [66]:
len(data.formula.unique())

2959302

In [56]:
alex_pbe = pd.read_csv("alexandria/convex_hull_pbe.csv")

In [62]:
len(alex_pbe.formula.unique())

144651

In [63]:
alex_pbe.columns

Index(['Unnamed: 0', 'mat_id', 'prototype_id', 'location', 'formula',
       'elements', 'spg', 'nsites', 'stress', 'energy_total', 'total_mag',
       'band_gap_ind', 'band_gap_dir', 'dos_ef', 'energy_corrected',
       'e_above_hull', 'e_form', 'e_phase_separation', 'decomposition',
       'in_nomad', 'cif'],
      dtype='object')

## Alchemy

чет как-то не пон как правильно сделать конвертацию из sdf в cif. Не понимаю откуда взять Lattice и переодичность

In [1]:
from pymatgen.core import Structure, Lattice
from pymatgen.entries.computed_entries import ComputedStructureEntry
import json 
import pandas as pd
from tqdm import tqdm
import os

In [2]:
pd.read_csv("alchemy-v20191129//final_version.csv")

,gdb_idx,atom number,"zpve\n(Ha, zero point vibrational energy)","Cv\n(cal/molK, heat capacity at 298.15 K)","gap\n(Ha, LUMO-HOMO)","G\n(Ha, Free energy at 298.15 K)","HOMO\n(Ha, energy of HOMO)","U\n(Ha, internal energy at 298.15 K)","alpha\n(a_0^3, Isotropic polarizability)","U0\n(Ha, internal energy at 0 K)","H\n(Ha, enthalpy at 298.15 K)","LUMO\n(Ha, energy of LUMO)","mu\n(D, dipole moment)","R2\n(a_0^2, electronic spatial extent)"
0,2859833,9,0.226164,0.000067,0.300531,-352.128828,-0.236722,-352.076085,99.201621,-352.088194,-352.075141,0.063809,0.180558,1919.249225
1,3148292,9,0.180782,0.000058,0.293098,-386.820766,-0.241536,-386.774917,84.670738,-386.784878,-386.773973,0.051562,1.302656,1420.242859
2,3607838,9,0.205922,0.000057,0.307544,-388.062328,-0.224965,-388.018758,86.198178,-388.028208,-388.017814,0.082579,1.284923,1257.238492
3,9540153,11,0.160845,0.000056,0.195923,-515.046508,-0.203538,-515.001198,95.801267,-515.010568,-515.000254,-0.007615,4.074910,1600.220066
4,340363,10,0.177712,0.000057,0.254495,-403.843164,-0.248943,-403.800346,95.147725,-403.809426,-403.799402,0.005552,4.111533,1442.171110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202574,9581016,10,0.120222,0.000051,0.179768,-450.915828,-0.244325,-450.872449,88.581216,-450.881390,-450.871505,-0.064557,6.434535,1541.115465
202575,9706527,10,0.130179,0.000053,0.189401,-438.632624,-0.231340,-438.589755,96.102362,-438.598674,-438.588811,-0.041938,0.338522,1628.839361
202576,9760179,10,0.119919,0.000051,0.163827,-450.920199,-0.232783,-450.875248,89.074794,-450.884399,-450.874304,-0.068956,4.337358,1497.024559
202577,9845175,10,0.119740,0.000050,0.191354,-454.722866,-0.251374,-454.680504,89.026059,-454.689065,-454.679560,-0.060020,5.263580,1611.219983


In [4]:
from rdkit import Chem

# Укажите путь к вашему .sdf файлу
sdf_file = 'D:\Projects\projects_python\genCry\\alchemy-v20191129\\atom_9\\2859833.sdf'
cif_file = 'file.cif'

# Считываем молекулы из .sdf файла
suppl = Chem.SDMolSupplier(sdf_file)

# Обрабатываем каждую молекулу и сохраняем в .cif файл
with open(cif_file, 'w') as f:
    for mol in suppl:
        if mol is not None:  # если молекула была загружена успешно
            # Преобразуем молекулу в CIF формат
            cif_data = Chem.MolToMolBlock(mol)
            f.write(cif_data + '\n')


In [8]:
from rdkit import Chem
from pymatgen.core import Structure

# Укажите путь к вашему .sdf файлу
sdf_file_path = 'D:\Projects\projects_python\genCry\\alchemy-v20191129\\atom_9\\2859833.sdf'

# Считываем молекулы из .sdf файла
suppl = Chem.SDMolSupplier(sdf_file_path)

# Обрабатываем каждую молекулу
for mol in suppl:
    if mol is not None:  # Если молекула была загружена успешно
        # Получаем атомы и координаты
        atomic_numbers = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
        coords = mol.GetConformer().GetPositions()

        # Создаем структуру pymatgen
        structure = Structure(lattice=None, species=atomic_numbers, coords=coords)

        # Выводим информацию о структуре
        print(structure)


ValueError: cannot reshape array of size 1 into shape (3,3)

In [9]:
pip install openbabel

  Using cached openbabel-3.1.1.1.tar.gz (82 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build openbabel
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for openbabel (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Guessing Open Babel location:
      - include_dirs: ['d:\\Projects\\projects_python\\genCry\\.venv\\include', 'C:\\Program Files\\Python311\\include', 'C:\\Program Files\\Python311\\Include', '/usr/local/include/openbabel3']
      - library_dirs: ['d:\\Projects\\projects_python\\genCry\\.venv\\libs', 'C:\\Program Files\\Python311\\libs', 'C:\\Program Files\\Python311', 'd:\\Projects\\projects_python\\genCry\\.venv\\PCbuild\\amd64', '/usr/local/lib']
      building 'openbabel._openbabel' extension
      swigging openbabel\openbabel-python.i to openbabel\openbabel-python_wrap.cpp
      swig.exe -python -c++ -small -O -templatereduce -naturalvar -Id:\Projects\projects_python\genCry\.venv\include "-IC:\Program Files\Python311\include" "-IC:\Program Files\Py

## Jarvis

In [18]:
from jarvis.db.figshare import data

dft_3d = data('dft_3d')

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:09<00:00, 4.42MiB/s]  


Loading the zipfile...
Loading completed.


In [19]:
df = pd.DataFrame(dft_3d)

In [24]:
def create_structure_from_data(data):
    """
    Создает объект Structure из предоставленных данных.

    :param data: dict, содержащий данные для создания структуры
    :return: pymatgen Structure
    """
    lattice_mat = data['lattice_mat']
    coords = data['coords']
    elements = data['elements']

    lattice = Lattice(lattice_mat)

    structure = Structure(lattice=lattice, species=elements, coords=coords)

    return structure

In [30]:
atoms = df['atoms']
cifs_for_dataset = []
for atom in tqdm(atoms):
    cifs_for_dataset.append(create_structure_from_data(atom).to_file(fmt='cif'))

  1%|          | 669/75993 [00:00<00:58, 1289.74it/s]d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\composition.py:1337: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda x: [get_el_sp(x).X, x])
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\composition.py:366: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  syms = sorted(sym_amt, key=lambda sym: get_el_sp(sym).X)
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\core\periodic_table.py:278: UserWarning: No Pauling electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  if not hasattr(other, "X") or not ha

In [ ]:
df_cif = pd.DataFrame({"cif": cifs_for_dataset})
final_df = pd.concat([df, df_cif], axis=1)
final_df.to_csv('Jarvis/jarvis.csv')

In [9]:
jarvis = pd.read_csv("Jarvis\\jarvis.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_18248\2014893053.py:1: DtypeWarning: Columns (14,17,26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  jarvis = pd.read_csv("Jarvis\\jarvis.csv")


In [10]:
jarvis

,Unnamed: 0,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,...,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search,cif
0,0,JVASP-90856,129,P4/nmm,TiCuSiAs,-0.42762,OptB88vdW,0.000,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0...",na,...,na,[],8,na,na,na,na,mp-1080455,-As-Cu-Si-Ti,# generated using pymatgen\ndata_TiCuSiAs\n_sy...
1,1,JVASP-86097,221,Pm-3m,DyB6,-0.41596,OptB88vdW,0.000,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]...",na,...,na,"['OPT-LOPTICS,JVASP-86097.zip,https://ndownloa...",7,na,na,na,na,mp-568319,-B-Dy,# generated using pymatgen\ndata_DyB6\n_symmet...
2,2,JVASP-64906,119,I-4m2,Be2OsRu,0.04847,OptB88vdW,0.000,"{'lattice_mat': [[-1.833590720595598, 1.833590...",na,...,na,"['OPT-LOPTICS,JVASP-64906.zip,https://ndownloa...",4,na,na,na,na,auid-3eaf68dd483bf4f4,-Be-Os-Ru,# generated using pymatgen\ndata_Be2OsRu\n_sym...
3,3,JVASP-98225,14,P2_1/c,KBi,-0.44140,OptB88vdW,0.472,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0...",na,...,na,[],32,na,na,na,na,mp-31104,-Bi-K,# generated using pymatgen\ndata_KBi\n_symmetr...
4,4,JVASP-10,164,P-3m1,VSe2,-0.71026,OptB88vdW,0.000,"{'lattice_mat': [[1.6777483798834445, -2.90594...",na,...,0.23,"['FD-ELAST,JVASP-10.zip,https://ndownloader.fi...",3,48.79,33.05,0.0,na,mp-694,-Se-V,# generated using pymatgen\ndata_VSe2\n_symmet...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75988,75988,JVASP-156020,139,I4/mmm,AcRh2Pb2,-0.30652,OptB88vdW,0.000,"{'lattice_mat': [[-2.374509402119681, 2.374509...",na,...,na,[],5,na,na,na,na,1206834,-Ac-Pb-Rh,# generated using pymatgen\ndata_Ac(RhPb)2\n_s...
75989,75989,JVASP-156398,216,F-43m,PrTlZn,-0.34112,OptB88vdW,0.000,"{'lattice_mat': [[-0.0, 3.4210598347774503, 3....",na,...,na,[],3,na,na,na,na,915022,-Pr-Tl-Zn,# generated using pymatgen\ndata_PrTlZn\n_symm...
75990,75990,JVASP-156099,139,I4/mmm,BaIn2Bi2,-0.39352,OptB88vdW,0.000,"{'lattice_mat': [[4.082347574975881, -4.076131...",na,...,0.67,[],5,30.67,-11.44,na,na,1214095,-Ba-Bi-In,# generated using pymatgen\ndata_Ba(InBi)2\n_s...
75991,75991,JVASP-156007,139,I4/mmm,TmSi2Tc2,-0.54853,OptB88vdW,0.000,"{'lattice_mat': [[2.90400678672412, -2.9037689...",na,...,na,[],5,na,na,na,na,1206745,-Si-Tc-Tm,# generated using pymatgen\ndata_Tm(SiTc)2\n_s...


In [11]:
jarvis.columns

Index(['Unnamed: 0', 'jid', 'spg_number', 'spg_symbol', 'formula',
       'formation_energy_peratom', 'func', 'optb88vdw_bandgap', 'atoms',
       'slme', 'magmom_oszicar', 'spillage', 'elastic_tensor',
       'effective_masses_300K', 'kpoint_length_unit', 'maxdiff_mesh',
       'maxdiff_bz', 'encut', 'optb88vdw_total_energy', 'epsx', 'epsy', 'epsz',
       'mepsx', 'mepsy', 'mepsz', 'modes', 'magmom_outcar', 'max_efg',
       'avg_elec_mass', 'avg_hole_mass', 'icsd', 'dfpt_piezo_max_eij',
       'dfpt_piezo_max_dij', 'dfpt_piezo_max_dielectric',
       'dfpt_piezo_max_dielectric_electronic',
       'dfpt_piezo_max_dielectric_ionic', 'max_ir_mode', 'min_ir_mode',
       'n-Seebeck', 'p-Seebeck', 'n-powerfact', 'p-powerfact', 'ncond',
       'pcond', 'nkappa', 'pkappa', 'ehull', 'Tc_supercon', 'dimensionality',
       'efg', 'xml_data_link', 'typ', 'exfoliation_energy', 'spg', 'crys',
       'density', 'poisson', 'raw_files', 'nat', 'bulk_modulus_kv',
       'shear_modulus_gv', 'mbj_ban

## AFlow

In [18]:
import json
from urllib.request import urlopen
from urllib.error import URLError, HTTPError

API = 'http://aflow.org/API/aflux/'
MATCHBOOK = 'species(Metals)'
DIRECTIVE = '$paging(1)'
REQUEST = API + '?' + MATCHBOOK + ',' + DIRECTIVE

try:
    with urlopen(REQUEST) as response:
        data = response.read().decode('utf-8')
        if data:
            response_json = json.loads(data)
            print(response_json)
        else:
            print("Пустой ответ от сервера")
except HTTPError as e:
    print(f"HTTP ошибка: {e.code} - {e.reason}")
except URLError as e:
    print(f"Ошибка URL: {e.reason}")
except json.JSONDecodeError as e:
    print(f"Ошибка декодирования JSON: {e}")

[{'compound': 'Ag2Al1As1', 'auid': 'aflow:01422465d644e394', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC011.ABC', 'spacegroup_relax': 12, 'Pearson_symbol_relax': 'mS8', 'species': ['Ag', 'Al', 'As']}, {'compound': 'Ag1Al2As1', 'auid': 'aflow:02a47840537cd60d', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC015.BCA', 'spacegroup_relax': 65, 'Pearson_symbol_relax': 'oS8', 'species': ['Ag', 'Al', 'As']}, {'compound': 'Ag1Al1As2', 'auid': 'aflow:05ee886b34da7910', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC014.CAB', 'spacegroup_relax': 10, 'Pearson_symbol_relax': 'mP4', 'species': ['Ag', 'Al', 'As']}, {'compound': 'Ag1Al2As1', 'auid': 'aflow:05e0175334714afb', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TBCC011.BCA', 'spacegroup_relax': 123, 'Pearson_symbol_relax': 'tP4', 'species': ['Ag', 'Al', 'As']}, {'compound': 'Ag2Al1As1', 'auid': 'aflow:06134d799be7badf', 'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC009.ABC', 'spacegroup_rel

In [19]:
response_json

[{'compound': 'Ag2Al1As1',
  'auid': 'aflow:01422465d644e394',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC011.ABC',
  'spacegroup_relax': 12,
  'Pearson_symbol_relax': 'mS8',
  'species': ['Ag', 'Al', 'As']},
 {'compound': 'Ag1Al2As1',
  'auid': 'aflow:02a47840537cd60d',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC015.BCA',
  'spacegroup_relax': 65,
  'Pearson_symbol_relax': 'oS8',
  'species': ['Ag', 'Al', 'As']},
 {'compound': 'Ag1Al1As2',
  'auid': 'aflow:05ee886b34da7910',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TFCC014.CAB',
  'spacegroup_relax': 10,
  'Pearson_symbol_relax': 'mP4',
  'species': ['Ag', 'Al', 'As']},
 {'compound': 'Ag1Al2As1',
  'auid': 'aflow:05e0175334714afb',
  'aurl': 'aflowlib.duke.edu:AFLOWDATA/LIB3_RAW/AgAlAs/TBCC011.BCA',
  'spacegroup_relax': 123,
  'Pearson_symbol_relax': 'tP4',
  'species': ['Ag', 'Al', 'As']},
 {'compound': 'Ag2Al1As1',
  'auid': 'aflow:06134d799be7badf',
  'aurl': 'aflowlib.duke.edu:AFLOWDA

In [20]:
len(response_json)

64

In [22]:
from aflow import *

result = search(batch_size=20
        ).select(K.agl_thermal_conductivity_300K
        ).filter(K.Egap > 6).orderby(K.agl_thermal_conductivity_300K, True)

# Now, you can just iterate over the results.
for entry in result:
    print(entry.Egap)

6.8531
8.183
7.4494
7.6056
7.0887
7.7111
8.0374
7.0234
7.4209
6.61
6.464
8.7442
6.5707
6.6252
6.8283
6.8386
7.1318
6.6538
6.9475
6.135
6.9841
6.93
6.8631
6.733
7.534
7.3543
6.5855
6.5663
6.2324
7.5731
6.7691
6.9891
6.6888
6.236
6.2916
6.9132
6.539
6.1381
6.6145
6.4219


KeyboardInterrupt: 

## perov_5

In [8]:
perov = pd.read_csv("perov_5\\perov_5_file.csv")
perov

,Unnamed: 0,material_id,cif,formula,heat_all,heat_ref,dir_gap,ind_gap
0,8989,6334,# generated using pymatgen\ndata_TlCoN2O\n_sym...,CoTlON2,2.72,2.624864,0.0,0.0
1,4450,17313,# generated using pymatgen\ndata_CrTeN3\n_symm...,CrTeN3,1.86,1.755497,0.0,0.0
2,13752,6840,# generated using pymatgen\ndata_SbTeN2O\n_sym...,SbTeON2,1.06,1.056506,0.0,0.0
3,12082,11379,# generated using pymatgen\ndata_NiPtO2F\n_sym...,PtNiO2F,1.52,1.513021,0.0,0.0
4,18849,16590,# generated using pymatgen\ndata_YGeN3\n_symme...,YGeN3,0.90,0.905283,0.0,0.0
...,...,...,...,...,...,...,...,...
18923,6903,6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...,CsNbON2,1.28,0.779384,0.0,0.0
18924,12146,7673,# generated using pymatgen\ndata_FePbN2O\n_sym...,PbFeON2,1.12,1.074549,0.0,0.0
18925,62,819,# generated using pymatgen\ndata_NaGeNO2\n_sym...,NaGeO2N,0.74,0.731097,0.0,0.0
18926,5519,11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...,TeAuO2F,1.76,1.750256,0.0,0.0


## carbon_24

In [13]:
carbon = pd.read_csv("carbon_24\\carbon_24_file.csv")
carbon

,Unnamed: 0,material_id,cif,energy_per_atom
0,9251,C-130499-1826-36,# generated using pymatgen\ndata_C\n_symmetry_...,-154.311891
1,6412,C-13904-4247-31,# generated using pymatgen\ndata_C\n_symmetry_...,-154.332183
2,3301,C-92138-4782-35,# generated using pymatgen\ndata_C\n_symmetry_...,-154.178157
3,4777,C-192672-505-73,# generated using pymatgen\ndata_C\n_symmetry_...,-154.309339
4,3764,C-193956-5355-22,# generated using pymatgen\ndata_C\n_symmetry_...,-154.141835
...,...,...,...,...
10148,6268,C-193926-6320-8,# generated using pymatgen\ndata_C\n_symmetry_...,-154.168573
10149,9375,C-13665-1988-16,# generated using pymatgen\ndata_C\n_symmetry_...,-154.184871
10150,542,C-142855-9438-3,# generated using pymatgen\ndata_C\n_symmetry_...,-154.096413
10151,8795,C-13640-2755-30,# generated using pymatgen\ndata_C\n_symmetry_...,-154.119650


In [14]:
print(carbon.iloc[0].cif)

# generated using pymatgen
data_C
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   2.48053000
_cell_length_b   5.09056000
_cell_length_c   4.89497000
_cell_angle_alpha   57.97521000
_cell_angle_beta   59.51109000
_cell_angle_gamma   43.04559000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   C
_chemical_formula_sum   C6
_cell_volume   34.99453900
_cell_formula_units_Z   6
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  C  C0  1  1.01706897  0.71072668  0.70544499  1
  C  C1  1  0.68025235  0.06989044  1.00318540  1
  C  C2  1  0.14234414  0.62471612  0.20939241  1
  C  C3  1  0.34433233  0.10871703  0.78081493  1
  C  C4  1  -0.11663885  0.55405962  0.57440517  1
  C  C5  1  0.00817595  0.46805771  0.07810120  1



## mp_20

In [15]:
mp = pd.read_csv("mp_20\\mp20_file.csv")
mp

,Unnamed: 0,material_id,formation_energy_per_atom,band_gap,pretty_formula,e_above_hull,elements,cif,spacegroup.number
0,37228,mp-1221227,-1.637460,0.2133,Na3MnCoNiO6,0.043001,"['Co', 'Mn', 'Na', 'Ni', 'O']",# generated using pymatgen\ndata_Na3MnCoNiO6\n...,8
1,19480,mp-974729,-0.314759,0.0000,Nd(Al2Cu)4,0.000000,"['Al', 'Cu', 'Nd']",# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...,139
2,29624,mp-1185360,-0.193761,0.0000,LiMnIr2,0.018075,"['Ir', 'Li', 'Mn']",# generated using pymatgen\ndata_LiMnIr2\n_sym...,225
3,38633,mp-1188861,-0.584694,3.8556,LiCSN,0.048847,"['C', 'Li', 'N', 'S']",# generated using pymatgen\ndata_LiCSN\n_symme...,62
4,10889,mp-677272,-2.474759,0.4707,La2EuS4,0.000000,"['Eu', 'La', 'S']",# generated using pymatgen\ndata_La2EuS4\n_sym...,122
...,...,...,...,...,...,...,...,...,...
45224,21009,mp-1023925,-1.152798,1.6970,WS2,0.001173,"['S', 'W']",# generated using pymatgen\ndata_WS2\n_symmetr...,164
45225,31626,mp-1187764,-0.788772,0.0000,Y2ZnPt,0.022813,"['Pt', 'Y', 'Zn']",# generated using pymatgen\ndata_Y2ZnPt\n_symm...,225
45226,20673,mp-1219588,-2.910913,1.9239,RbMgCoF6,0.000710,"['Co', 'F', 'Mg', 'Rb']",# generated using pymatgen\ndata_RbMgCoF6\n_sy...,74
45227,5349,mp-3589,-2.764644,7.2758,BPO4,0.000000,"['B', 'P', 'O']",# generated using pymatgen\ndata_BPO4\n_symmet...,82


In [16]:
len(set(mp.material_id) & set(jarvis.reference))

27624

## mpts_52

In [19]:
mpts = pd.read_csv("mpts_52\\mpts_52.csv")
mpts

,Unnamed: 0,material_id,theoretical,energy_above_hull,formation_energy_per_atom,year,cif
0,66,mp-66,False,0.136413,0.136413,1913,# generated using pymatgen\ndata_C\n_symmetry_...
1,22862,mp-22862,False,0.000000,-2.109608,1913,# generated using pymatgen\ndata_NaCl\n_symmet...
2,30,mp-30,False,0.000000,0.000000,1914,# generated using pymatgen\ndata_Cu\n_symmetry...
3,2741,mp-2741,False,0.000000,-4.224281,1914,# generated using pymatgen\ndata_CaF2\n_symmet...
4,1455,mp-1455,False,0.110733,-0.778788,1914,# generated using pymatgen\ndata_MnS2\n_symmet...
...,...,...,...,...,...,...,...
40471,11924,mp-11924,False,0.000000,-1.204892,2004,# generated using pymatgen\ndata_Rb2TaCuSe4\n_...
40472,11775,mp-11775,False,0.000000,-3.304561,2004,# generated using pymatgen\ndata_RbUO3\n_symme...
40473,31118,mp-31118,False,0.000000,-3.921264,2004,# generated using pymatgen\ndata_SmScO3\n_symm...
40474,11778,mp-11778,False,0.000000,-1.492701,2004,# generated using pymatgen\ndata_ThMnSe3\n_sym...


In [23]:
len(set(mp.material_id) | set(mpts.material_id) | set(jarvis.reference))

102518

## Merger

Возьмем датасеты mp_20 mpts_52 carbon_24 jarvis alex_pbe perov_5. Сделаем объединение всех датасетов в один большой, где общий признаки будут cif файл + material_id 

In [70]:
jarvis = pd.read_csv("Jarvis\\jarvis.csv")
mpts = pd.read_csv("mpts_52\\mpts_52.csv")
mp = pd.read_csv("mp_20\\mp20_file.csv")
carbon = pd.read_csv("carbon_24\\carbon_24_file.csv")
alex_pbe = pd.read_csv("alexandria\\convex_hull_pbe.csv")
perov = pd.read_csv("perov_5/perov_5_file.csv")

C:\Users\user\AppData\Local\Temp\ipykernel_18248\4050457690.py:1: DtypeWarning: Columns (14,17,26,30) have mixed types. Specify dtype option on import or set low_memory=False.
  jarvis = pd.read_csv("Jarvis\\jarvis.csv")


### jarvis -> jarvis_cry

In [71]:
jarvis_cry = jarvis[[ "reference", "cif"]]

In [72]:
jarvis_cry.columns = ["material_id", "cif"]

In [73]:
jarvis_cry

,material_id,cif
0,mp-1080455,# generated using pymatgen\ndata_TiCuSiAs\n_sy...
1,mp-568319,# generated using pymatgen\ndata_DyB6\n_symmet...
2,auid-3eaf68dd483bf4f4,# generated using pymatgen\ndata_Be2OsRu\n_sym...
3,mp-31104,# generated using pymatgen\ndata_KBi\n_symmetr...
4,mp-694,# generated using pymatgen\ndata_VSe2\n_symmet...
...,...,...
75988,1206834,# generated using pymatgen\ndata_Ac(RhPb)2\n_s...
75989,915022,# generated using pymatgen\ndata_PrTlZn\n_symm...
75990,1214095,# generated using pymatgen\ndata_Ba(InBi)2\n_s...
75991,1206745,# generated using pymatgen\ndata_Tm(SiTc)2\n_s...


### mpts_52 -> mpts_cry

In [74]:
mpts_cry = mpts[["material_id", "cif"]]

In [37]:
print(mpts_cry.iloc[3].cif)

# generated using pymatgen
data_CaF2
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   3.90043777
_cell_length_b   3.90043777
_cell_length_c   3.90043777
_cell_angle_alpha   60.00000000
_cell_angle_beta   60.00000000
_cell_angle_gamma   60.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   CaF2
_chemical_formula_sum   'Ca1 F2'
_cell_volume   41.95899366
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Ca  Ca0  1  0.00000000  0.00000000  0.00000000  1
  F  F1  1  0.25000000  0.25000000  0.25000000  1
  F  F2  1  0.75000000  0.75000000  0.75000000  1



В mpts есть кристалы, которые состоят из одного атома, это надо пофиксить

### mp_20 -> mp_cry

In [75]:
mp_cry = mp[['material_id', "cif"]]
mp_cry

,material_id,cif
0,mp-1221227,# generated using pymatgen\ndata_Na3MnCoNiO6\n...
1,mp-974729,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...
2,mp-1185360,# generated using pymatgen\ndata_LiMnIr2\n_sym...
3,mp-1188861,# generated using pymatgen\ndata_LiCSN\n_symme...
4,mp-677272,# generated using pymatgen\ndata_La2EuS4\n_sym...
...,...,...
45224,mp-1023925,# generated using pymatgen\ndata_WS2\n_symmetr...
45225,mp-1187764,# generated using pymatgen\ndata_Y2ZnPt\n_symm...
45226,mp-1219588,# generated using pymatgen\ndata_RbMgCoF6\n_sy...
45227,mp-3589,# generated using pymatgen\ndata_BPO4\n_symmet...


### carbon_24 -> carbon_cry

In [76]:
carbon_cry = carbon.drop(["energy_per_atom", 'Unnamed: 0'], axis=1)

In [77]:
carbon_cry

,material_id,cif
0,C-130499-1826-36,# generated using pymatgen\ndata_C\n_symmetry_...
1,C-13904-4247-31,# generated using pymatgen\ndata_C\n_symmetry_...
2,C-92138-4782-35,# generated using pymatgen\ndata_C\n_symmetry_...
3,C-192672-505-73,# generated using pymatgen\ndata_C\n_symmetry_...
4,C-193956-5355-22,# generated using pymatgen\ndata_C\n_symmetry_...
...,...,...
10148,C-193926-6320-8,# generated using pymatgen\ndata_C\n_symmetry_...
10149,C-13665-1988-16,# generated using pymatgen\ndata_C\n_symmetry_...
10150,C-142855-9438-3,# generated using pymatgen\ndata_C\n_symmetry_...
10151,C-13640-2755-30,# generated using pymatgen\ndata_C\n_symmetry_...


### alexandria_pbe -> alex_pbe_cry

In [68]:
alex_pbe_cry = alex_pbe[["mat_id", "cif"]]

In [87]:
alex_pbe_cry.columns = ["material_id", "cif"]

In [88]:
alex_pbe_cry

,material_id,cif
0,agm048224876,# generated using pymatgen\ndata_Al4IrRu3\n_sy...
1,agm002302240,# generated using pymatgen\ndata_EuAgTeO\n_sym...
2,agm003161479,# generated using pymatgen\ndata_CaTlAg\n_symm...
3,agm003330027,# generated using pymatgen\ndata_Tm3Ge3Ir2\n_s...
4,agm002198109,# generated using pymatgen\ndata_EuAl4Pd\n_sym...
...,...,...
144646,agm005618834,# generated using pymatgen\ndata_Sm3Ga2Ir7\n_s...
144647,agm003208591,# generated using pymatgen\ndata_Li(Fe3Ge2)2\n...
144648,agm005866520,# generated using pymatgen\ndata_Nb12Sn3W\n_sy...
144649,agm003157526,# generated using pymatgen\ndata_CeP\n_symmetr...


### perov_5 -> perov_cry

In [80]:
perov_cry = perov[["material_id", "cif"]]

In [82]:
perov_cry['material_id'] = 'p-' + perov_cry['material_id'].astype(str)

C:\Users\user\AppData\Local\Temp\ipykernel_18248\2236623699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  perov_cry['material_id'] = 'p-' + perov_cry['material_id'].astype(str)


In [83]:
perov_cry

,material_id,cif
0,p-6334,# generated using pymatgen\ndata_TlCoN2O\n_sym...
1,p-17313,# generated using pymatgen\ndata_CrTeN3\n_symm...
2,p-6840,# generated using pymatgen\ndata_SbTeN2O\n_sym...
3,p-11379,# generated using pymatgen\ndata_NiPtO2F\n_sym...
4,p-16590,# generated using pymatgen\ndata_YGeN3\n_symme...
...,...,...
18923,p-6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...
18924,p-7673,# generated using pymatgen\ndata_FePbN2O\n_sym...
18925,p-819,# generated using pymatgen\ndata_NaGeNO2\n_sym...
18926,p-11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...


### concat

In [92]:
genCry = pd.concat([mp_cry, mpts_cry, jarvis_cry, alex_pbe_cry, carbon_cry, perov_cry], ignore_index=True)

In [93]:
genCry = genCry.drop_duplicates()

In [97]:
genCry = genCry.reset_index().drop("index", axis=1)
genCry

,material_id,cif
0,mp-1221227,# generated using pymatgen\ndata_Na3MnCoNiO6\n...
1,mp-974729,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...
2,mp-1185360,# generated using pymatgen\ndata_LiMnIr2\n_sym...
3,mp-1188861,# generated using pymatgen\ndata_LiCSN\n_symme...
4,mp-677272,# generated using pymatgen\ndata_La2EuS4\n_sym...
...,...,...
315444,p-6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...
315445,p-7673,# generated using pymatgen\ndata_FePbN2O\n_sym...
315446,p-819,# generated using pymatgen\ndata_NaGeNO2\n_sym...
315447,p-11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...


Так как alexandria_pbe имеет только уникальные формулы, то значит они отрелаксированные и мы можем их использовать. Значит добавим их тоже в датасет

Тут сохраняем предварительную версию. Надо пробежаться и убрать кристаллы из одного атома

In [98]:
genCry.to_csv("genCry_dataset/genCry.csv")

## Val

In [3]:
genCry = pd.read_csv("genCry_dataset\\genCry.csv")

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.io.cif import CifParser
from io import StringIO

def filter_structures_from_csv(dataframe, num_of_atoms = 2):
    """
    Читает CSV файл с CIF строками, фильтрует структуры с более чем 2 атомами
    и сохраняет результаты в новый CSV файл.

    :param input_csv_path: Путь к входному CSV файлу.
    :param output_csv_path: Путь к выходному CSV файлу.
    """    
    filtered_data = []

    for index, row in tqdm(dataframe.iterrows()):
        mat_id = row["material_id"]
        cif_string = row['cif'] 
        
        try:
            cif_file_like = StringIO(cif_string)
            
            parser = CifParser(cif_file_like)
            structure = parser.get_structures()[0]
            
            if len(structure) > num_of_atoms:
                filtered_data.append({
                    'material_id': mat_id,
                    'cif': cif_string
                })
        except Exception as e:
            print(f"Ошибка при обработке строки {index}: {e}")
    
    filtered_df = pd.DataFrame(filtered_data)
    return filtered_df

In [8]:
genCry_small = filter_structures_from_csv(genCry)

0it [00:00, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_14084\2410766679.py:26: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures()[0]
1it [00:00,  2.32it/s]d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\io\cif.py:1347: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  return self.parse_structures(*args, **kwargs)
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\io\cif.py:1347: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  return self.parse_structures(*args, **kwargs)
d:\Projects\projects_python\genCr

Ошибка при обработке строки 74238: Invalid CIF file with no structures!


98866it [08:06, 272.79it/s]

Ошибка при обработке строки 98821: Invalid CIF file with no structures!


102152it [08:17, 336.41it/s]d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\io\cif.py:1312: UserWarning: Some occupancies ([2.0, 2.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
  if struct := self._get_structure(data, primitive, symmetrized, check_occu=check_occu):
d:\Projects\projects_python\genCry\.venv\Lib\site-packages\pymatgen\io\cif.py:1347: UserWarning: Issues encountered while parsing CIF: Some occupancies ([2.0, 2.0]) sum to > 1! If they are within the occupancy_tolerance, they will be rescaled. The current occupancy_tolerance is set to: 1.0
No structure parsed for section 1 in CIF.
Occupancy 2.0 exceeded tolerance.
  return self.parse_structures(*args, **kwargs)
102187it [08:17, 335.05it/s]

...

279053it [23:56, 197.84it/s]

Ошибка при обработке строки 279024: Invalid CIF file with no structures!


282816it [24:15, 198.06it/s]

Ошибка при обработке строки 282787: Invalid CIF file with no structures!


315449it [26:37, 197.44it/s]


In [10]:
genCry_small.to_csv("genCry_dataset\\genCry_s.csv")

In [11]:
genCry_small

,material_id,cif
0,mp-1221227,# generated using pymatgen\ndata_Na3MnCoNiO6\n...
1,mp-974729,# generated using pymatgen\ndata_Nd(Al2Cu)4\n_...
2,mp-1185360,# generated using pymatgen\ndata_LiMnIr2\n_sym...
3,mp-1188861,# generated using pymatgen\ndata_LiCSN\n_symme...
4,mp-677272,# generated using pymatgen\ndata_La2EuS4\n_sym...
...,...,...
308226,p-6452,# generated using pymatgen\ndata_CsNbN2O\n_sym...
308227,p-7673,# generated using pymatgen\ndata_FePbN2O\n_sym...
308228,p-819,# generated using pymatgen\ndata_NaGeNO2\n_sym...
308229,p-11261,# generated using pymatgen\ndata_TeAuO2F\n_sym...
